##### 问题:
我们需要创建一个实例，但是出于某些原因想绕过__init__()方法，用别的方式来创建。

##### 解决方案:
可以直接调用类的__new__()方法来创建一个未初始化的实例。例如，考虑下面这
个类：

In [8]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day 

采用下面的方法可以在不调用__init__()的情况下创建一个 Date 实例：

In [9]:
d = Date.__new__(Date) 
print(d)
# print(d.year) #AttributeError: 'Date' object has no attribute 'year'

可以看到，得到的实例是未经初始化的。因此，给实例变量设定合适的初始值现在就
成了我们的责任。示例如下：


In [10]:
data = {'year':2012, 'month':8, 'day':29}
for key, value in data.items():
    setattr(d, key, value)
print(d.year) 
print(d.month)

2012
8


当需要以非标准的方式来创建实例时常常会遇到需要绕过__init__()的情况。比如反序
列化（deserializing）数据，或者实现一个类方法将其作为备选的构造函数，都属于这
种情况。例如，在前面给出的 Date 类中，有人可能会定义一个可选的构造函数 today()：

In [ ]:
from time import localtime
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    @classmethod
    def today(cls):
        d = cls.__new__(cls)
        t = localtime()
        d.year = t.tm_year
        d.month = t.tm_mon
        d.day = t.tm_mday
        return d

类似地，假设正在反序列化 JSON 数据，要产生一个下面这样的字典：

In [ ]:
data = { 'year': 2012, 'month': 8, 'day': 29 }

如果想将这个字典转换为一个 Date 实例，只要使用解决方案中给出的技术即可。


当需要以非标准的方式创建实例时，通常最好不要对它们的实现做过多假设。因此，
一般来说不要编写直接操纵底层实例字典__dict__的代码，除非能保证它已被定义。否
则，如果类中使用了__slots__、property 属性、描述符或者其他高级技术，那么代码就
会崩溃。通过使用 setattr()来为属性设定值，代码就会尽可能的通用。